# This notebook automatically reads in a retrieval summary file and computes the output plots and statistics.
### Evert Nasedkin
### nasedkinevert@gmail.com

It's optimised for retrievals with a particular naming and directory structure, but should be fairly easily generalisable.
The important parameters to update are the retrieval list (list of all retrieval names), data_dir (where the retrievals are stored) and the obs_dir (where the observations are stored).

In [1]:
import sys,os
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from spectres import spectres
from astropy.io import fits
import scicomap as sc
import json 

from petitRADTRANS.retrieval import plot_style as ps
from petitRADTRANS.retrieval.parameter import Parameter
from petitRADTRANS.retrieval import RetrievalConfig, Retrieval, plot_style
from petitRADTRANS.retrieval.util import gaussian_prior, inverse_gamma_prior
from petitRADTRANS.retrieval.models import  *
from petitRADTRANS import nat_cst as nc
from molmass import Formula
from petitRADTRANS.retrieval.util import *
import copy as cp

model_function_dictionary = {}
model_function_dictionary["emission_model_diseq"] = emission_model_diseq
model_function_dictionary["emission_model_diseq_simple_patchy_clouds"] = emission_model_diseq_simple_patchy_clouds
model_function_dictionary["interpolated_profile_emission"] = interpolated_profile_emission
model_function_dictionary["guillot_emission"] = guillot_emission
model_function_dictionary["gradient_profile_emission"] = gradient_profile_emission
model_function_dictionary["gradient_constrained"] = gradient_constrained

model_function_dictionary["guillot_patchy_emission"] = guillot_patchy_emission
model_function_dictionary["guillot_transmission"] = guillot_transmission
model_function_dictionary["guillot_patchy_transmission"] = guillot_patchy_transmission
model_function_dictionary["isothermal_transmission"] = isothermal_transmission
model_function_dictionary["guillot_patchy_transmission_constrained_chem"] = guillot_patchy_transmission_constrained_chem


map_538 = sc.ScicoQualitative(cmap='538')
fixed_cmap_538 = map_538.get_mpl_color_map()
colour_dict = {"b":fixed_cmap_538(3/6),
              "c":fixed_cmap_538(2/6),
              "d":fixed_cmap_538(1/6),
              "e":fixed_cmap_538(0/6)}

Using pRT Plotting style!


Let's add a couple useful functions for calculating the metallicity from the mass fraction abundances.

In [2]:

def calc_feh_from_abundances(samples, line_species):
    solar_nfracs = {}
    solar_nfracs['H'] = 0.9207539305712663
    solar_nfracs['He'] = 0.07836886940606574
    solar_nfracs['C'] = 0.0002478241000191816
    solar_nfracs['N'] = 6.225060569980629e-05
    solar_nfracs['O'] = 0.00045096580003490476
    solar_nfracs['Na'] = 1.6000869436558967e-06
    solar_nfracs['Mg'] = 3.665587420837336e-05
    solar_nfracs['Al'] = 2.5950000002008533e-06
    solar_nfracs['Si'] = 2.979500000230613e-05
    solar_nfracs['P'] = 2.3667020201598627e-07
    solar_nfracs['S'] = 1.2137900735543473e-05
    solar_nfracs['Cl'] = 2.911679585221254e-07
    solar_nfracs['K'] = 9.866056120020401e-08
    solar_nfracs['Ca'] = 2.0143901144484638e-06
    solar_nfracs['Ti'] = 8.206228044298752e-08
    solar_nfracs['V'] = 7.836886941506495e-09
    solar_nfracs['Fe'] = 2.911679585221254e-05
    solar_nfracs['Ni'] = 1.5280711681810828e-06


    """solar_mass_fracs = {}
    norm = 0.
    for species in solar_nfracs.keys():
        mass = Formula(species).mass
        solar_mass_fracs[species] = solar_nfracs[species]*mass
        norm += solar_nfracs[species]*mass

    for species in solar_nfracs.keys():
        solar_mass_fracs[species] = solar_mass_fracs[species]/norm"""

    mass_fractions = {}
    msum = 0
    for spec in line_species:
        mass_fractions[spec] = samples[spec]
        msum += 10**samples[spec]
       
    if "H2" in list(samples.keys()):
        mass_fractions["H2"] = samples["H2"]
    else:
        mass_fractions["H2"] = 0.766 * (1.0-msum)
    if "He" in list(samples.keys()):
        mass_fractions["He"] = samples["He"]
    else:
        mass_fractions["He"] = 0.234 * (1.0-msum)
    n_fracs = mass_to_number(mass_fractions)

    atom_abunds = solar_nfracs['C'] + solar_nfracs['O']
    
    CNO_metals = 0.0
    if 'H2O_Exomol' in list(samples.keys()):
        CNO_metals = CNO_metals + n_fracs['H2O_Exomol']

    if 'CH4' in list(samples.keys()):
        CNO_metals = CNO_metals + n_fracs['CH4'] 

    if 'NH3' in list(samples.keys()):
        CNO_metals = CNO_metals +  n_fracs['NH3']
        atom_abunds = atom_abunds + solar_nfracs['N']

    if 'CO_all_iso_HITEMP' in list(samples.keys()):
        CNO_metals = CNO_metals + n_fracs['CO_all_iso_HITEMP']

    if 'CO2' in list(samples.keys()):
        CNO_metals = CNO_metals + (3* n_fracs['CO2'])

    if 'SO2' in list(samples.keys()):
        CNO_metals = CNO_metals + (3* n_fracs['SO2'])
        atom_abunds = atom_abunds + solar_nfracs['S']

    metallicity = np.log10(CNO_metals/atom_abunds) 
    #samples["Fe/H"] = metallicity
    return metallicity
test = calc_feh_from_abundances({"H2":np.array([0.75]), "He":np.array([0.23]), "H2O_Exomol":np.array([0.1])},["H2O_Exomol"])
print(test)

[1.25885168]


Set up the directories here. Obviously this works on my own laptop and needs to be adjusted to your own directory structure.

In [3]:
obs_dir = "/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/observations/"
base_dir="/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/"
#obs_dir = "/Users/nasedkin/Documents/RetrievalResults/WASP39/observations/"
#base_dir="/Users/nasedkin/Documents/RetrievalResults/WASP39/"
#planet_dirs = [base_dir]
planet_dirs = sorted(glob(base_dir + "*free"))
planet_dirs.extend(sorted(glob(base_dir + "*full")))
print(planet_dirs)


['/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/b_free', '/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/c_free', '/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/d_free', '/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/e_free', '/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/b_full', '/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/c_full', '/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/d_full', '/Users/nasedkin/Documents/RetrievalResults/HR8799_2023_Retrievals/e_full']


Ok, now let's iterate over all of the retrievals in the base_dir.
We're going to read in the summary file to set up the data and parameters, then generate the standard plots and compute some statistics about the retrieval.

In [4]:
ignore_list = ["HR8799b_23_v01_free_all_correctos_freeMgCloud_guillot_newphot_mrprior",
               "HR8799b_23_v02_free_all_correctos_freeMgCloud_grad_newphot_mrprior",
               "HR8799b_23_v03_free_all_correctos_freeMgCloud_grad_newphot_mrprior",
               #"HR8799c_23_v03_free_all_onlygrav_nogpi_eqMgCloud_guillot",
               #"HR8799c_23_v03_free_all_all_noos_freeMgCloud_guillot",
               #"HR8799d_23_v04_free_all_eqFeMgCloud_guillot_fseds",
               #"HR8799c_23_v03_free_all_all_noos_freeMgCloud_guillot",
               #"HR8799d_23_v01_diseq_all_eqFeMgCloud_fseds_newconv_newphot",
               "HR8799c_23_v04_free_grav_noos_freeFeMgCloud_spline_newphot_newGPI_fseds_mr",
               "HR8799c_23_v01_diseq_all_grav_freeFeMgCloud_logg_fseds_newbins_newphot",
               #"HR8799c_23_v03_free_nograv_os_freeFeMgCloud_guillot_newphot",
               #"HR8799c_23_v04_free_grav_noos_freeFeMg2Cloud_guillot_newphot_newGPI_fseds_mr",
               #"HR8799d_23_v04_free_all_freeFeMgCloud_guillot",
               #"HR8799e_23_v01_free_all_freeAl2Cloud_guillot_newphot_newconv",
               #"HR8799d_23_v04_free_all_freeMgFeCloud_spline_scaleALES_newSPHGPI",
               #"HR8799d_23_v04_free_all_eqFeMgCloud_guillot_fseds",
               "HR8799d_23_v04_free_all_eqFeMgCloud_guillot_fseds",
               "HR8799e_23_v01_free_all_freeSiCCloud_guillot_newphot_newconv",
               "HR8799c_23_v03_free_all_onlygrav_nogpi_eqMgCloud_guillot",
               "HR8799b_23_v03_diseq_all_scaleos_eqFeMgCloud_mrprior_highfeh_spline_fseds",
               #"HR8799e_23_v04_free_all_freeFeMgCloud_guillotpatchy2_newphot_newconv_newSPHGPI",
               #"HR8799b_23_v03_diseq_all_scaleos_eqFeMgCloud_mrprior_highfeh_spline_fseds",
               "WASP39b_Eureka_diseq"] # Leftovers from an old setup, don't worry about this
for data_dir in planet_dirs:
    if not data_dir.endswith("/"):
        data_dir += "/"
    eval_dirs = sorted(glob(data_dir + "evaluate*"))
    retrievals = [retrieval.split("evaluate_")[-1] for retrieval in eval_dirs]
    for ret in retrievals:
        #if not "HR8799e_23_v02_free_all_freeFeMgCloud_grad_fseds_newphot_newconv_newSPHGPI" in ret:
        #    continue
        if "HR8799b" in ret and "scaleos" in ret and not "diseq" in ret:
            continue
        if ret in ignore_list:
            continue
        #if os.path.exists(data_dir + "evaluate_" + ret + "/" +ret + "_ShortStats.txt"):
        #    continue
        if not os.path.exists(data_dir + "out_PMN/"+ ret + "_post_equal_weights.dat"):
            continue
        test = np.genfromtxt(data_dir + "out_PMN/"+ ret + "_post_equal_weights.dat")
        print(test.shape)
        if len(test)<30:
            continue
        print(ret)
        amr = False
        scattering = True
        pressures = np.logspace(-6,3,80)
        if "clear" in ret:
            amr = False
            scattering = False
            pressures = np.logspace(-6,3,100)
        RunDefinition = RetrievalConfig(retrieval_name = ret, # Give a useful name for your retrieval
                                        run_mode = "evaluate", # 'retrieval' to run, or 'evaluate' to make plots
                                        pressures = pressures,
                                        AMR = amr,             # Adaptive mesh refinement, slower if True
                                        scattering = scattering)      # Add scattering for emission spectra clouds
        try:
            sumfile = list(open(f"{data_dir}evaluate_{ret}/{ret}_ret_summary.txt", 'r'))
        except:
            continue
        data_start = False
        free_param_start = False
        fixed_param_start = False

        data_dict = {}
        free_param_list = []
        fixed_param_dict = {}
        current_name = None
        fail = False
        with open(f"{data_dir}out_PMN/{ret}_params.json", 'r') as f:
            free_param_list = json.load(f)
        for i,line in enumerate(sumfile):
            if line.strip() == "":
                continue
            if line.strip() == "Data":
                data_start = True
                free_param_start = False
                fixed_param_start = False
                continue
            if "Multinest Outputs" in line:
                data_start = False
                continue
            if "Best Fit" in line:
                data_start = False
            if "Free Parameters" in line:
                free_param_start = True
                fixed_param_start = False
                continue
            if "Fixed Parameters" in line:
                fixed_param_start = True
                continue

            if free_param_start:
                continue

            if fixed_param_start:
                param, val = line.split("=")
                param = param.strip()
                if param == "nnodes":
                    fixed_param_dict[param] = int(float(val.replace('\n',"").strip()))
                    continue
                if param == "num_layer":
                    fixed_param_dict[param] = int(float(val.replace('\n',"").strip()))
                    continue
                if param == "N_layers":
                    fixed_param_dict[param] = int(float(val.replace('\n',"").strip()))
                    continue
                fixed_param_dict[param] = float(val.replace('\n',"").strip())
                continue

            if data_start:
                if not line[0]==" ":
                    instrument = line.replace('\n',"")
                    data_dict[instrument] = {}
                    data_dict[instrument]["path"] = None
                    data_dict[instrument]["data resolution"] = None
                    data_dict[instrument]["model resolution"] = None
                    data_dict[instrument]["scale factor"] = False
                    data_dict[instrument]["scale err"] = False
                    data_dict[instrument]["offset"] = False
                    data_dict[instrument]["photometry"] = False
                    data_dict[instrument]["external_pRT_reference"] = None
                    if f"{instrument}_offset" in free_param_list:
                        data_dict[instrument]["offset"] = True
                    current_name = instrument
                    continue
                #if "HR8799_2023_Retrievals" in line:
                if "observations" in line:
                    obsfile = line.split("observations/")[-1].strip()
                    data_dict[current_name]["path"] = obs_dir + obsfile
                    if not os.path.exists(obs_dir + obsfile):
                        fail = True
                    continue
                param,val = line.split("=")
                param = param.strip()
                param = param.replace('\n',"")
                if param == "Model Function":
                    data_dict[current_name][param] = model_function_dictionary[val.strip()]
                    continue
                if param == "photometric width":
                    low,high = val.split("--")
                    low = float(low)
                    high = float(high.split("um")[0])
                    data_dict[current_name][param] = [low,high]
                    continue
                if param == "Photometric transform function":
                    data_dict[current_name]["photometry"] = True
                    continue
                if param == "model resolution":
                    data_dict[current_name][param] = int(val.strip())
                    continue
                if "scale" in param:
                    data_dict[current_name]["scale factor"] = True
                    data_dict[current_name]["scale err"] = True
                    continue
                if "offset" in param:
                    data_dict[current_name]["offset"] = True
                    continue
                if "external" in param:
                    data_dict[current_name]["external_pRT_reference"] = val.replace('\n',"").strip()
                    continue
                data_dict[current_name][param] = float(val.strip())
        phot_added = False
        if fail:
            continue
        for key, val in data_dict.items():
            if not 'Model Function' in val.keys():
                print(key,val)
                continue
            if not val["photometry"]:
                distance = 10*nc.pc
                if key == "GRAVITY" or key=="ALES":
                    distance = 41.2925*nc.pc
                RunDefinition.add_data(key,
                                    val["path"],
                                    data_resolution = val["data resolution"],
                                    external_pRT_reference = val['external_pRT_reference'],
                                    distance = distance,
                                    scale = val["scale factor"],
                                    scale_err = val["scale err"],
                                    offset_bool= val["offset"],
                                    model_generating_function = val["Model Function"])
                if distance != 10*nc.pc:
                    RunDefinition.data[key].scale_to_distance(10.0*nc.pc)
                if key == "OSIRIS2011":
                    RunDefinition.data['OSIRIS2011'].flux = 3e-12*RunDefinition.data['OSIRIS2011'].flux*1e-3/(RunDefinition.data['OSIRIS2011'].wlen)**2
                    RunDefinition.data['OSIRIS2011'].flux_error = 3e-12*RunDefinition.data['OSIRIS2011'].flux_error*1e-3/(RunDefinition.data['OSIRIS2011'].wlen)**2
                    RunDefinition.data['OSIRIS2011'].covariance = None
                if key == "OSIRIS" and "HR8799c" in ret:
                    continue
                    newwlens = RunDefinition.data['OSIRIS'].wlen[1:-4:6]
                    newflux, newerr = spectres(newwlens,
                                            RunDefinition.data['OSIRIS'].wlen,
                                            RunDefinition.data['OSIRIS'].flux,
                                            RunDefinition.data['OSIRIS'].flux_error,
                                            fill=np.nan)

                    RunDefinition.data['OSIRIS'].wlen = newwlens[~np.isnan(newflux)]
                    RunDefinition.data['OSIRIS'].flux = newflux[~np.isnan(newflux)]
                    RunDefinition.data['OSIRIS'].flux_error = np.sqrt(newflux[~np.isnan(newflux)])

                    RunDefinition.data['OSIRIS'].wlen_bins = np.zeros_like(RunDefinition.data['OSIRIS'].wlen)
                    RunDefinition.data['OSIRIS'].wlen_bins[:-1] = np.diff(RunDefinition.data['OSIRIS'].wlen)
                    RunDefinition.data['OSIRIS'].wlen_bins[-1] = RunDefinition.data['OSIRIS'].wlen_bins[-2]
                    RunDefinition.data['OSIRIS'].update_covariance_from_flux_error()
                if key == "SPHEREYJH" and "HR8799d" in ret:
                    sphere_data_cut =-5
                    RunDefinition.data["SPHEREYJH"].flux = RunDefinition.data["SPHEREYJH"].flux[:sphere_data_cut]
                    RunDefinition.data["SPHEREYJH"].wlen = RunDefinition.data["SPHEREYJH"].wlen[:sphere_data_cut]
                    RunDefinition.data["SPHEREYJH"].flux_error = RunDefinition.data["SPHEREYJH"].flux_error[:sphere_data_cut]
                    RunDefinition.data["SPHEREYJH"].covariance = RunDefinition.data["SPHEREYJH"].covariance[:sphere_data_cut,:sphere_data_cut]
                    RunDefinition.data["SPHEREYJH"].inv_cov = np.linalg.inv(RunDefinition.data["SPHEREYJH"].covariance)
                    sign, RunDefinition.data["SPHEREYJH"].log_covariance_determinant = np.linalg.slogdet(2.0 * np.pi * RunDefinition.data["SPHEREYJH"].covariance)
                    RunDefinition.data["SPHEREYJH"].wlen_bins = np.zeros_like(RunDefinition.data["SPHEREYJH"].wlen)
                    RunDefinition.data["SPHEREYJH"].wlen_bins[:-1] = np.diff(RunDefinition.data["SPHEREYJH"].wlen)
                    RunDefinition.data["SPHEREYJH"].wlen_bins[-1] = RunDefinition.data["SPHEREYJH"].wlen_bins[-2]
            else:
                if not phot_added:
                    RunDefinition.add_photometry(val["path"],
                                        model_generating_function = val["Model Function"],
                                        model_resolution = 40 )
                    phot_added = True
        for key, val in fixed_param_dict.items():
            RunDefinition.add_parameter(name = key, free = False, value = val)

        #################################################
        # Add parameters, and priors for free parameters.
        #################################################
        lines_use = []
        #lines = ['H2O_Exomol', 'CO_all_iso_HITEMP','CH4', 'CO2', "C2H2", 'HCN', 'NH3','H2S','Na_allard', 'K_allard']
        lines = ['H2O_Exomol', 'CO_all_iso_HITEMP', 'CH4', 'CO2', 'NH3', 'HCN', 'H2S', 'FeH', 'PH3', 'Na_allard', 'K_allard', 'TiO_all_Exomol', 'VO','SiO']
        if "HR8799b" in ret:
            lines = ['H2O_Exomol','CO_all_iso_HITEMP', 'CH4','CO2', 'HCN', 'H2S', "NH3"]        
        for param in free_param_list:
            if "eq_scaling" in param:
                continue
            if "log_X_cb" in param:
                continue
            if "log_Pbase" in param:
                continue
            if "SO2" in param:
                continue
            if param in lines:
                lines_use.append(param)
                continue
            RunDefinition.add_parameter(param, True,transform_prior_cube_coordinate = lambda x : x)
        if "SO2" in free_param_list:
            lines_use.append("SO2")
        #######################################################
        # Define species to be included as absorbers
        #######################################################
        RunDefinition.set_rayleigh_species(['H2', 'He'])
        RunDefinition.set_continuum_opacities(['H2-H2', 'H2-He'])    
        if not "C/O" in free_param_list:
            if ret == "WASP39b_Full_guillot_free_grey_R300_precomputed_lowH2O":
                lines_use.remove("H2O_Exomol")
                lines_use.remove("SO2")
                lines_use.append("H2O_Exomol")
                lines_use.append("SO2")
            RunDefinition.set_line_species(lines_use, eq=False)
        else:
            RunDefinition.set_line_species(lines,eq=True)
            for spec in lines_use:
                RunDefinition.add_line_species(spec,eq=False)
        clouds = []
        for param in free_param_list:
            eq = False
            if "eq_scaling" in param:
                eq = True
                tag = "_cd"
                if "Mg" in param:
                    if "amCloud" in ret:
                        tag="_ad"
                    if ret == "HR8799e_23_v02_diseq_all_eqFeMgCloud_newphot_newSPHGPI_mrprior_grad_fseds_hansen":
                        tag = "_ad"
                RunDefinition.add_cloud_species(f"{param.split('_')[-1]}{tag}",eq = eq,scaling_factor = (-2.5,4.5))
                continue
            if "log_X_cb" in param:
                print(f"Adding cloud {param}")
                tag = "_cd"
                cname = param.split("log_X_cb_")[-1]
                if "Mg" in param:
                    if "amCloud" in ret:
                        tag="_ad"
                    if ret == "HR8799e_23_v02_diseq_all_eqFeMgCloud_newphot_newSPHGPI_mrprior_grad_fseds_hansen":
                        tag = "_ad"
                base = None
                if f"log_Pbase_{cname}" in free_param_list or "SiC" in cname or "Al2" in cname:
                    base = (-6,3)
                #RunDefinition.add_cloud_species(f"{param.split('_')[-1]}{tag}", eq = False, abund_lim = (-6.5,0.0), PBase_lim=(-6,3))
                print(f"Cloud Base {cname}= {base}")
                RunDefinition.add_cloud_species(f"{param.split('_')[-1]}{tag}", eq = eq, abund_lim = (-6.5,0.0), PBase_lim=base)
                continue

        for key, value in RunDefinition.parameters.items():
            value.plot_in_corner = True
            value.corner_label = key.replace('_',' ')


        ##################################################################
        # Define axis properties of spectral plot if run_mode == 'evaluate'
        ##################################################################
        RunDefinition.plot_kwargs["spec_xlabel"] = 'Wavelength [micron]'

        RunDefinition.plot_kwargs["spec_ylabel"] = "Flux, 10pc [W/m2/micron]"
        RunDefinition.plot_kwargs["y_axis_scaling"] = 1
        RunDefinition.plot_kwargs["xscale"] = 'linear'
        RunDefinition.plot_kwargs["yscale"] = 'linear'
        RunDefinition.plot_kwargs["resolution"] = None#80.
        RunDefinition.plot_kwargs["nsample"] = 400.

        ##################################################################
        # Define from which observation object to take P-T
        # in evaluation mode (if run_mode == 'evaluate'),
        # add PT-envelope plotting options
        ##################################################################
        if "GRAVTY" in data_dict.keys():
            RunDefinition.plot_kwargs["take_PTs_from"] = 'GRAVITY'
        else:
            RunDefinition.plot_kwargs["take_PTs_from"] = list(data_dict.keys())[0]
        RunDefinition.plot_kwargs["temp_limits"] = [50, 3000]
        RunDefinition.plot_kwargs["press_limits"] = [1e2, 1e-6]

        ##################################################################
        # Run the Retrieval
        ##################################################################
        retrieval = Retrieval(RunDefinition,
                            output_dir = data_dir,
                            sample_spec = False,
                            ultranest = False,
                            test_plotting = False)
        #retrieval.plot_data()

        try:
            retrieval.run(n_live_points = 1000,
                    sampling_efficiency=0.8,
                    const_efficiency_mode=False,
                    resume = True)
        except FileNotFoundError as error:
            print(error)
            print(f"Could not load retrieval {ret}")
            continue
        #retrieval.plot_all(contribution = True, mode = 'median')
        sample_dict, parameter_dict = retrieval.get_samples(retrieval.output_dir)
        samples_use = cp.copy(sample_dict[retrieval.retrieval_name])
        if samples_use.shape[0]<10:
            print(f"Not enough samples in {ret}")
            continue

        parameters_read = parameter_dict[retrieval.retrieval_name]
        #try: 
        #    retrieval.plot_spectra(samples_use, parameters_read, refresh=True, mode = "median")
        #except ValueError as error:
        #    print(error)
        #    print(f"Failed to plot the spectrum of {retrieval.retrieval_name}")
        #    ignore_list.append(retrieval.retrieval_name)
        #    continue

        #for key, val in retrieval.best_fit_params.items():
        #    print(key,val.value)

        planet = "e"
        if not os.path.exists(retrieval.output_dir + "evaluate_" + ret + "/" + ret + "_PT_envelopes.pdf"):
            retrieval.plot_PT(sample_dict, parameters_read, contribution = True, refresh=False, mode = "median")
        samples_small = {}
        inds = np.random.randint(0,samples_use.shape[0],1000)
        samples_small[retrieval.retrieval_name] = samples_use[inds]
        """if len(parameters_read) < 17:
            retrieval.plot_corner(samples_small, 
                                parameter_dict, 
                                parameters_read, 
                                plot_best_fit=True, 
                                true_values = None,
                                contour_kwargs={"linewidths":3, "alpha" : 1.0},
                                hist_kwargs = {"linewidth":3,"histtype":"stepfilled", "ec":'k',"alpha":0.8},
                                hist2d_kwargs={},
                                colors = [colour_dict[planet]])"""
        #retrieval.plot_contribution(samples_use, parameters_read, refresh=False, mode = "median")
        #retrieval.plot_abundances(samples_use, parameters_read, refresh=False, mode = "median")
        #retrieval.plot_spectra(samples_use, parameters_read, refresh=True, mode = "bestfit")
        if not os.path.exists(retrieval.output_dir + "evaluate_" + ret + "/" + ret +"_volume_mixing_ratio_profiles.npy"):
            retrieval.save_volume_mixing_ratios(sample_dict, parameter_dict)

        fehs = []
        if not "diseq" in retrieval.retrieval_name:
            for sample in samples_small[retrieval.retrieval_name]:
                abunds, mmw = retrieval.get_mass_fractions(sample,parameters_read)
                fehs.append(calc_feh_from_abundances(abunds, lines_use)[0])
        fehs = np.array(fehs)
        #if "HR8799e_23_v02_free_all_freeFeMgCloud_grad_fseds_newphot_newconv_newSPHGPI" in ret:
        retrieval.sample_teff(sample_dict, parameter_dict, nsample = 1000)

        logl,bfind = retrieval.get_best_fit_likelihood(samples_use)
        bf_pars = retrieval.get_max_likelihood_params(samples_use[bfind],parameters_read)
        chi2 = retrieval.get_chi2(samples_use[bfind])
        norm = retrieval.get_chi2_normalisation(samples_use[bfind])
        red_chi2 = retrieval.get_reduced_chi2(samples_use[bfind],subtract_n_parameters=True)
        chi2_ndata = retrieval.get_reduced_chi2(samples_use[bfind],subtract_n_parameters=False)
        evidence = retrieval.get_evidence()
        myfile = retrieval.output_dir + "evaluate_" + ret + "/" + ret + "_ShortStats.txt"
        print(myfile)
        with open(myfile, 'w') as f:
            f.write(f"#{retrieval.retrieval_name}\n")
            f.write(f"ll, {logl}\n")
            f.write(f"chi2, {chi2}\n")
            f.write(f"norm, {norm}\n")
            f.write(f"chi/ndata, {chi2_ndata}\n")
            f.write(f"chi2/nu, {red_chi2}\n")
            f.write(f"logZ, {evidence}\n")
            if "free" in retrieval.retrieval_name:
                f.write(f"[M/H], {np.median(fehs)}+-{np.std(fehs)}")
        #plt.close('all')



(38741, 13)
HR8799b_23_v01_free_all_correctos_clear_0nodes_v2


/Users/nasedkin/anaconda3/envs/pRTUpdate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


species v0.6.0
Working folder: /Users/nasedkin/Documents/Paper2_HR8799_Notebooks
 -> A new version (0.7.4) is available!
 -> It is recommended to update to the latest version
 -> See https://github.com/tomasstolker/species for details
Configuration settings:
   - Database: /Users/nasedkin/Documents/Paper2_HR8799_Notebooks/species_database.hdf5
   - Data folder: /Users/nasedkin/Documents/Paper2_HR8799_Notebooks/data
   - Interpolation method: linear
   - Magnitude of Vega: 0.03
Starting retrieval HR8799b_23_v01_free_all_correctos_clear_0nodes_v2
Setting up PRT Objects


/Users/nasedkin/python-packages/petitRADTRANS/petitRADTRANS/radtrans.py:119: FutureWarning: pRT_input_data_path was set by an environment variable. In a future update, the path to the petitRADTRANS input_data will be set within a .ini file that will be automatically generated into the user home directory (OS agnostic), inside a .petitradtrans directory
  warnings.warn(f"pRT_input_data_path was set by an environment variable. In a future update, the path to "


  Read line opacities of H2O_Exomol...
 Done.
  Read line opacities of CO_all_iso_HITEMP...
 Done.
  Read line opacities of CH4...
 Done.
  Read line opacities of CO2...
 Done.
  Read line opacities of HCN...
 Done.
  Read line opacities of H2S...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.

  Read line opacities of H2O_Exomol...
 Done.
  Read line opacities of CO_all_iso_HITEMP...
 Done.
  Read line opacities of CH4...
 Done.
  Read line opacities of CO2...
 Done.
  Read line opacities of HCN...
 Done.
  Read line opacities of H2S...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.

  Read line opacities of H2O_Exomol_R_40...
 Done.
  Read line opacities of CO_all_iso_HITEMP_R_40...
 Done.
  Read line opacities of CH4_R_40...
 Done.
  Read line opacities of CO2_R_40...
 Done.
  Read line opacities of HCN_R_40...
 Done.
  Read line opacities of H2S_R_40...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opaciti

KeyboardInterrupt: 